In [1]:
import mne
from glob import glob
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session

In [2]:
all_files_path=glob('../data/*.edf')
print(len(all_files_path))
healthy_file_path=[i for i in all_files_path if  'H' in i.split('\\')[1]]
patient_file_path=[i for i in all_files_path if  'M' in i.split('\\')[1]]
print(len(healthy_file_path),len(patient_file_path))
def read_data(file_path):
    raw = mne.io.read_raw_edf(file_path, preload=True)
    raw.pick_types(meg=False, eeg=True, eog=False, ecg=False) # Selecting EEG, EOG and ECG channels
    # Select a specific channel
    channel_to_keep = ['EEG Fp1-LE', 'EEG F3-LE', 'EEG C3-LE', 'EEG P3-LE', 'EEG O1-LE', 'EEG F7-LE', 'EEG T3-LE', 'EEG T5-LE', 'EEG Fz-LE', 'EEG Fp2-LE', 'EEG F4-LE', 'EEG C4-LE', 'EEG P4-LE', 'EEG O2-LE', 'EEG F8-LE', 'EEG T4-LE', 'EEG T6-LE', 'EEG Cz-LE', 'EEG Pz-LE']  
    # Replace with the name of the channel you want to keep
    raw.pick_channels(channel_to_keep)
    raw.set_eeg_reference()
    raw.filter(l_freq=30,h_freq=100)#1-4=delta, 4-8=theta, 8-12=alpha, 12-30=beta, 30-100=gamma
    epochs=mne.make_fixed_length_epochs(raw,duration=15,overlap=1)
    epochs=epochs.get_data()
    scaler = StandardScaler()
    data = scaler.fit_transform(epochs.reshape(-1,epochs.shape[-1])).reshape(epochs.shape)
    return data #trials,channel,length

162
76 86


In [3]:
%%capture
control_epochs_array=[read_data(subject) for subject in healthy_file_path]
patients_epochs_array=[read_data(subject) for subject in patient_file_path]
control_epochs_labels=[len(i)*[0] for i in control_epochs_array]
patients_epochs_labels=[len(i)*[1] for i in patients_epochs_array]

In [4]:
data_list=control_epochs_array+patients_epochs_array
label_list=control_epochs_labels+patients_epochs_labels
groups_list=[[i]*len(j) for i, j in enumerate(data_list)]

data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(groups_list)

data_array=np.moveaxis(data_array,1,2)
X=data_array
y=label_array

In [5]:
def cnnmodel():
    clear_session()
    model=Sequential()
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(3840,19)))#1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#2
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#6
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#8
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())#10
    model.add(Dense(1,activation='sigmoid'))#11
    
    model.compile('adam',loss='binary_crossentropy',metrics=['Accuracy', 'Precision', 'Recall','AUC'])
    return model

In [6]:
def load_data(num_clients,X,y):
    clients_X, clients_y = np.array_split(X, num_clients), np.array_split(y, num_clients)
    return clients_X, clients_y

In [7]:
def train_local_model(model, data_X, data_y):
    model.compile('adam',loss='binary_crossentropy',metrics=['Accuracy', 'Precision', 'Recall','AUC'])
    model.fit(data_X,data_y,epochs=25,batch_size=32)
    return model

In [8]:
# Initialize global model
global_model = cnnmodel()

# Number of devices
num_devices = 10
total_sample=0

# Number of communication rounds
num_communication_rounds = 1

clients_X, clients_y=load_data(num_devices,X,y)

In [9]:
# Federated Learning
for round in range(num_communication_rounds):
    local_models = []

    # Communication Round
    for i in range(num_devices):
        # Load data on each device
        local_data_X = clients_X[i]
        local_data_y=clients_y[i]

        # Clone the global model for each device
        local_model = tf.keras.models.clone_model(global_model)
        local_model.set_weights(global_model.get_weights())

        # Train the local model on local data
        # local_model.compile('adam',loss='binary_crossentropy',metrics=['Accuracy', 'Precision', 'Recall','AUC'])
        local_model = train_local_model(local_model, local_data_X, local_data_y)
        local_model.save('models/local_model_%s.h5' % i)
        # Save the local model for aggregation
        local_models.append((local_model, len(local_data_y)))

    # Aggregation (FedAvg)
    global_weights_sum = [tf.zeros_like(w) for w in global_model.get_weights()]

    for local_model, num_samples_i in local_models:
        local_weights = local_model.get_weights()
        total_sample+=num_samples_i
        weighted_local_weights = [num_samples_i * w for w in local_weights]
        global_weights_sum = [tf.add(gw, wlw) for gw, wlw in zip(global_weights_sum, weighted_local_weights)]

    # Compute the average weights
    average_weights = [tf.divide(gws, total_sample) for gws in global_weights_sum]

    # Update the global model with the average weights
    global_model.set_weights(average_weights)

    # Send the updated global model to all devices
    # for i in range(num_devices):
    #     send_model_update_to_server(global_model)

# The final global model
# final_model = global_model


Epoch 1/25
15/15 [==============================] - 8s 80ms/step - loss: 0.7164 - Accuracy: 0.4009 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.0000e+00
Epoch 2/25
15/15 [==============================] - 1s 78ms/step - loss: 0.5528 - Accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.0000e+00
Epoch 3/25
15/15 [==============================] - 1s 78ms/step - loss: 0.3865 - Accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.0000e+00
Epoch 4/25
15/15 [==============================] - 1s 78ms/step - loss: 0.2077 - Accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.0000e+00
Epoch 5/25
15/15 [==============================] - 1s 72ms/step - loss: 0.0815 - Accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.0000e+00
Epoch 6/25
15/15 [==============================] - 1s 73ms/step - loss: 0.0302 - Accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.0000e+00
Epoch 7/25
15/15 [==========

c:\Users\rahat\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/25
15/15 [==============================] - 4s 63ms/step - loss: 0.7152 - Accuracy: 0.3815 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.0000e+00
Epoch 2/25
15/15 [==============================] - 1s 63ms/step - loss: 0.5583 - Accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.0000e+00
Epoch 3/25
15/15 [==============================] - 1s 63ms/step - loss: 0.4018 - Accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.0000e+00
Epoch 4/25
15/15 [==============================] - 1s 64ms/step - loss: 0.2255 - Accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.0000e+00
Epoch 5/25
15/15 [==============================] - 1s 68ms/step - loss: 0.0911 - Accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.0000e+00
Epoch 6/25
15/15 [==============================] - 1s 67ms/step - loss: 0.0313 - Accuracy: 1.0000 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.0000e+00
Epoch 7/25
15/15 [==========

In [10]:
global_model.save('models/global_model.h5')